In [1]:
import osmnx as ox
import networkx as nx
import pandas as pd
import json
import requests

from rtree import index
from shapely.geometry import box
from utils import *

In [2]:
G = ox.save_load.load_graphml(filename="Singapore_drive_processed.graphml")
print(type(G))

<class 'networkx.classes.multidigraph.MultiDiGraph'>


In [3]:
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

In [6]:
nodes.head()

,y,x,osmid,highway,geometry,ref
1820000257,1.356580,103.932616,1820000257,traffic_signals,POINT (103.93262 1.35658),NaN
3874553858,1.319182,103.921237,3874553858,NaN,POINT (103.92124 1.31918),NaN
1820000259,1.356647,103.932745,1820000259,traffic_signals,POINT (103.93275 1.35665),NaN
3874553857,1.319219,103.921661,3874553857,NaN,POINT (103.92166 1.31922),NaN
2083586053,1.400663,103.872050,2083586053,NaN,POINT (103.87205 1.40066),NaN


In [30]:
# Make Singapore_node.tntp
f = open("data/Singapore_node.tntp", 'a')
f.write("Node\tX\tY\t;\n")

for n in nodes.values:
    f.write("{}\t{}\t{}\t;\n".format(n[2], n[1], n[0]))
f.close()

In [32]:
f = open("../../Traffic speed bands/Fri Feb 14 09_06_16 2020.json", "r").read()
x = json.loads(f)

speed_bands = pd.DataFrame.from_dict(x, orient='columns')
process_speed_band(speed_bands)

/home/orionstar25/Semester-8-Major-Project/Open Street Maps/Phase 4 - Modified Hybrid Search + CSO/utils.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MaximumSpeed'][i] = '80'


In [33]:
calculate_congestion(G, edges, speed_bands)
edges.head()

,u,v,key,osmid,oneway,name,highway,maxspeed,length,travel_time,Location,geometry,observed_speed
0,1820000257,1865254946,0,174765824,True,Tampines Avenue 8,tertiary,60.0,14.248,0.85488,"(1.3565805, 103.9326163, 1.3566943, 103.9326752)","LINESTRING (103.93262 1.35658, 103.93268 1.35669)",29.000000
1,1820000257,4662056865,0,587044201,True,Tampines Avenue 5,primary,60.0,27.282,1.63692,"(1.3565805, 103.9323996, 1.3566955, 103.9326163)","LINESTRING (103.93262 1.35658, 103.93240 1.35670)",27.333333
2,3874553858,6995781882,0,384254693,False,Fidelio Street,tertiary,40.0,20.740,1.8666,"(1.3191823, 103.9212368, 1.3192215, 103.9214192)","LINESTRING (103.92124 1.31918, 103.92142 1.31922)",27.333333
3,3874553858,3874553861,0,384254631,True,Fidelio Street,tertiary,40.0,15.191,1.3671900000000001,"(1.3191111, 103.9211207, 1.3191823, 103.9212368)","LINESTRING (103.92124 1.31918, 103.92112 1.31911)",240.000000
4,1820000259,245190289,0,170861145,True,Tampines Avenue 8,tertiary,60.0,13.573,0.8143800000000001,"(1.3565404, 103.9326858, 1.3566471, 103.9327451)","LINESTRING (103.93275 1.35665, 103.93269 1.35654)",240.000000


In [39]:
capacity = []

length = edges['length']
o_sp = edges['observed_speed']
m_sp = edges['maxspeed'].astype('float')
key = edges['key']

for l, o, m, k in zip(length, o_sp, m_sp, key):
    c = math.ceil(((3*(1+k) * l)/6) * (o/m))
    capacity.append(c)
edges['capacity'] = capacity

edges.head()

,u,v,key,osmid,oneway,name,highway,maxspeed,length,travel_time,Location,geometry,observed_speed,capacity
0,1820000257,1865254946,0,174765824,True,Tampines Avenue 8,tertiary,60.0,14.248,0.85488,"(1.3565805, 103.9326163, 1.3566943, 103.9326752)","LINESTRING (103.93262 1.35658, 103.93268 1.35669)",29.000000,4
1,1820000257,4662056865,0,587044201,True,Tampines Avenue 5,primary,60.0,27.282,1.63692,"(1.3565805, 103.9323996, 1.3566955, 103.9326163)","LINESTRING (103.93262 1.35658, 103.93240 1.35670)",27.333333,7
2,3874553858,6995781882,0,384254693,False,Fidelio Street,tertiary,40.0,20.740,1.8666,"(1.3191823, 103.9212368, 1.3192215, 103.9214192)","LINESTRING (103.92124 1.31918, 103.92142 1.31922)",27.333333,8
3,3874553858,3874553861,0,384254631,True,Fidelio Street,tertiary,40.0,15.191,1.3671900000000001,"(1.3191111, 103.9211207, 1.3191823, 103.9212368)","LINESTRING (103.92124 1.31918, 103.92112 1.31911)",240.000000,46
4,1820000259,245190289,0,170861145,True,Tampines Avenue 8,tertiary,60.0,13.573,0.8143800000000001,"(1.3565404, 103.9326858, 1.3566471, 103.9327451)","LINESTRING (103.93275 1.35665, 103.93269 1.35654)",240.000000,28


In [60]:
# Make Singapore_net.tntp
f = open("data/Singapore_net.tntp", 'a')
f.write("<NUMBER OF ZONES> 23219 \n<NUMBER OF NODES> 23219 \n<FIRST THRU NODE> 1 \n<NUMBER OF LINKS> 44612 \n<END OF METADATA> \n\n")
f.write("~\tInit node\tTerm node\tCapacity\tLength\tFree Flow Time\tB\tPower\tSpeed limit\tToll\tType\t;\n")

for e in edges.values:
    f.write("\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t;\n".format(e[0], e[1], e[13], e[8], e[9], 0.15, 4, e[7], 0, 1))

f.close()

In [17]:
# Make Singapore_trips.tntp
from random import randint

f = open("data/Singapore_trips.tntp", 'a')
f.write("<NUMBER OF ZONES> 23219 \n<TOTAL OD FLOW> \n<END OF METADATA> \n\n")
demand = 0

for n in nodes.values[:10]:
    f.write("Origin\t{}\t\n".format(n[2]))
    count = 0
    
    for m in nodes.values[-20:]:
        if m[2] == n[2]:
            f.write("\t\t{} : {};".format(m[2], 0))
        else:
            d = randint(0,5)
            f.write("\t\t{} : {};".format(m[2], d))
            demand = demand + d
            
        count = count + 1
        if count%5 == 0:
            f.write("  \n")
            
    f.write("\n\n")

f.close()
demand

480